In [ ]:
import intake
import xarray as xr
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
from dask.distributed import Client
client = Client(threads_per_worker=1)
client

In [ ]:
%%time
catalog_path = '/g/data/v14/tm4888/code/BRAN2020-intake-catalog/catalogs/'
BRAN2020_catalog = intake.open_esm_datastore(catalog_path+'BRAN2020.json', columns_with_iterables=['variable']) # load catalogue
search_uv_month = BRAN2020_catalog.search(variable=['u','v'],time_period='month') # search and filter data by variables and time period
DS = search_uv_month.to_dask() # load data lazily with Dask
DS_slice = DS.sel(st_ocean= slice(0,300)).sel(xu_ocean=slice(130,250)).sel(yu_ocean=slice(-30,30)).mean('st_ocean') # slice out reduced XYZ subset required
clim_uv = DS_slice.groupby('Time.month').mean(method="cohorts", engine="flox") ## calculate climatology & chunking rules everything!
speed = np.sqrt(clim_uv.u**2 + clim_uv.v**2) # calculate current speeds
speed.sel(month=8).plot(robust=True) #plot
plt.title('BRAN2020 current speed\n August climatology')

# compute uv climatology over upper 300m 

In [ ]:
%%time
clim_uv = clim_uv.compute()

In [ ]:
%%time
speed = speed.compute()

# write out to scratch

In [ ]:
PICOSOM_BRAN2020_uv_speed = xr.merge([speed.rename('speed'),clim_uv])

In [ ]:
PICOSOM_BRAN2020_uv_speed

In [ ]:
%%time
PICOSOM_BRAN2020_uv_speed.to_netcdf('/scratch/xv83/tm4888/PICOSOM_BRAN2020/PICOSOM_BRAN2020_uv_speed.nc')

# read in from nc file

In [ ]:
PICOSOM_BRAN2020_uv_speed_ = xr.open_dataset('/scratch/xv83/tm4888/PICOSOM_BRAN2020/PICOSOM_BRAN2020_uv_speed.nc')

In [ ]:
PICOSOM_BRAN2020_uv_speed_

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
import matplotlib.ticker as ticker
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy
from matplotlib import mlab, cm, gridspec
import matplotlib.ticker as mticker
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cmocean
%matplotlib inline

In [ ]:
def plot_currents_PlateCarree(central_longitude=180,figsize=(16,16),extent=[130, 290, -60, 30],
                    xticks=[150, 180, 210,240,270],yticks=[-30, -20,-10, 0, 10,20, 30],
                    xstride=10,ystride=10,my_plot_title='my_plot_title',optional_plot_data=None,
                    cmap='Spectral_r',
                    cbar_kwargs = None,
                    **kwargs):
    """
    plot_PlateCarree()
    - Make a geographic plot with the PlateCarree projection.
    - This function is tailored for "pacific centred" plots but could be used more generally.

    :param central_longitude: central_longitude value - default is 180
    :type central_longitude: int or float
    :param figsize: "figsize" kwarg for `plt.subplots`
    :type figsize: tuple of ints or floats
    :param extent: geographical extent of plot boundaries
    :type extent: list of 4 ints or floats as [x1,x2,y1,y2]
    :param xticks: list of x tick labels to draw
    :type xticks: list of ints or floats or empty list
    :param yticks: list of y tick labels to draw
    :type yticks: list of ints or floats or empty list
    :param xstride: stride of longitude grid in degrees - default is 10
    :type xstride: int
    :param ystride: stride of longitude grid in degrees - default is 10
    :type ystride: int
    :param optional_plot_data: xarray object to plot in 2D
    :type optional_plot_data: 2D xarray object
    :return: plot
    :rtype: plot
    :raises TypeError: TBD

    :typical use: plot_PlateCarree(optional_plot_data=plot_data,x='longitude',y='latitude',
    :levels=100,cbar_kwargs={'orientation':'horizontal', 'shrink':0.6, "pad" : .05, 'aspect':40},cmap = cmocean.cm.thermal,robust=True,vmin=0, vmax=31)
    """
    long_list = np.arange(-180, 180, xstride)
    lat_list = np.arange(-90, 90, ystride)
                        
    proj = ccrs.PlateCarree(central_longitude=central_longitude)

    fig, ax = plt.subplots(subplot_kw=dict(projection=proj), figsize=figsize)
    #fig.canvas.draw()
    
    resolution='50m'
    land = cartopy.feature.NaturalEarthFeature('physical', 'land', \
        scale=resolution, edgecolor='k', facecolor=cfeature.COLORS['land'])
    ax.add_feature(land)


    gl = ax.gridlines(crs=proj, draw_labels=False, alpha=0.3, linewidth=0.5)
    gl.xlocator = mticker.FixedLocator(long_list)
    gl.ylocator = mticker.FixedLocator(lat_list)

    ax.set_xticks(xticks, crs=ccrs.PlateCarree())
    ax.set_yticks(yticks, crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    if optional_plot_data is not None:
        ##### plotting #####
        optional_plot_data.plot.contourf(ax=ax, transform=ccrs.PlateCarree(), cbar_kwargs=cbar_kwargs, **kwargs, 
                                robust=True,cmap = cmap,add_colorbar=False)
    #plot u/v vectors
    # Define the x and y coordinates
    x = PICOSOM_BRAN2020_uv_speed_.xu_ocean
    y = PICOSOM_BRAN2020_uv_speed_.yu_ocean
    ax.quiver(x.values,y.values,u.values,v.values,transform=ccrs.PlateCarree(), 
              units='x', width=0.01, scale=0.7, headwidth=5,alpha=0.2)
    ax.set_extent(extent, crs=ccrs.PlateCarree())
    # Hide the axis label
    ax.set_ylabel('')
    ax.set_xlabel('')
    plt.title(my_plot_title)
    #plt.show()
    plot_DIR = '/scratch/xv83/tm4888/PICOSOM_BRAN2020/'
    plot_file_name = 'PICOSOM_BRAN2020_'+month_str+'_'+region_str+'.png'
    plot_path = plot_DIR + plot_file_name
    fig.tight_layout()
    plt.savefig(plot_path, dpi = 600, bbox_inches='tight')

In [ ]:
%%time
time_choice = 1
u = PICOSOM_BRAN2020_uv_speed_.u.sel(month=time_choice)
v = PICOSOM_BRAN2020_uv_speed_.v.sel(month=time_choice)
speed = PICOSOM_BRAN2020_uv_speed_.speed.sel(month=time_choice)
month_list = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
extent_PAC = [130,250,-30,30]
extent_Fiji = [160,190,-25,-10]
extent_PNG = [130,160,-15,5]
extent_Palau = [130,160,0,20]
extent_Marshalls = [165,210,-5,20]
month_str = month_list[time_choice-1]
plot_currents_PlateCarree(optional_plot_data=speed,extent=extent_PAC,levels=75,my_plot_title='BRAN2020 reanalysis\n upper 300m currents\n'+month_str+' climatology')

In [ ]:
month_list = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
extent_PAC = [130,250,-30,30]
extent_Fiji = [160,190,-25,-10]
extent_PNG = [130,160,-15,5]
extent_Palau = [130,160,0,20]
extent_Marshalls = [165,210,-5,20]
regions = ['Pacific','Fiji','PNG','Palau','Marshalls']
extent_dict = {'Pacific':extent_PAC,'Fiji':extent_Fiji,'PNG':extent_PNG,'Palau':extent_Palau,'Marshalls':extent_Marshalls}

In [ ]:
extent_dict['Pacific']

In [ ]:
%%time
for region_str in regions:
    region_extent = extent_dict[region_str]
    for time_choice in range(1,13):
        u = PICOSOM_BRAN2020_uv_speed_.u.sel(month=time_choice)
        v = PICOSOM_BRAN2020_uv_speed_.v.sel(month=time_choice)
        speed = PICOSOM_BRAN2020_uv_speed_.speed.sel(month=time_choice)
        month_str = month_list[time_choice-1]
        plot_currents_PlateCarree(optional_plot_data=speed,
                                  extent=region_extent,levels=75,
                                  my_plot_title='BRAN2020 reanalysis\n upper 300m currents\n'+month_str+' climatology')